In [1]:
from selenium import webdriver;
from bs4 import BeautifulSoup;
from selenium.webdriver.chrome.service import Service;
from selenium.webdriver.common.by import By;
from selenium.webdriver.chrome.options import Options;
from selenium.webdriver.common.action_chains import ActionChains;
from webdriver_manager.chrome import ChromeDriverManager;
import time;
import pandas as pd;
import re;

In [2]:
# *open window

options  = Options();
options.add_experimental_option("detach", True);
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options);

driver.get('https://www.dotproperty.com.ph/apartments-for-rent'); # *link to open
driver.maximize_window(); # *maximize the window

# data to get : price, name, location, accomodation, desc

# ! step 1 : get max page and curr page

soup = BeautifulSoup(driver.page_source, "html.parser");

last_page = ''
rows = []
cnt = 0
while last_page != 'disabled':
    # print('while:')
    last_page = driver.find_element(By.CSS_SELECTOR, 'ul.pagination > :last-child').get_attribute('class');
    # print('last page' , last_page);

    for item in soup.select('.search-list > .wrapper'):
        # print('for:' ,cnt, ":",item)
        price = item.select_one(".description-block > a > .price").text.strip();
        name = item.select_one(".description-block > a > .name").text.strip();
        location = item.select_one(".description-block > a > .location > small").text.strip();
        # accomodation = item.select_one("div.description-block > a > div.accommodation").text.strip();
        desc = item.select_one(".description-block > a > .description-text").text.strip();

        accomodation_holder = item.select("div.description-block > a > div.accommodation > ul");
        z = [];
        for x in accomodation_holder: # looping through feautures
            # print(x)
            try:
                ammenity =  str(x.select_one('li > i'));
                # get_attribute('class').replace('icon','')
                amount = x.select_one("span").text.strip();
                # print(ammenity, " : ", amount)
                y = ammenity + " : " + amount;
                # print(y)
                z.append(y);
            except :
                print('error')
        accomodation = z 
        # print('appending to rows')
        rows.append([name,location,desc,accomodation,price]) # appending to rows
        # rows

        cnt = cnt + 1
        # print(item)

    
   
    if(last_page == 'disabled'):
        print('finished')
        break
    else:
        time.sleep(2);
        actions = ActionChains(driver);

        toAction = driver.find_element(By.CLASS_NAME,"search-tools");
        actions.move_to_element(toAction).perform();

        time.sleep(2);
        actions = ActionChains(driver);

        nextbtn = driver.find_element(By.CSS_SELECTOR,"ul.pagination > :last-child > a");
        actions.click(nextbtn); 
        actions.perform();

        # curr_page = curr_page + 1;
        time.sleep(5);
        
        


# print(cnt)
rows


finished


[['2-Bedroom Furnished Apartment for Rent in Banawa, Cebu City McDonald',
  'Cebu City\n                                    , Cebu',
  'Property Type: Apartment for RentLocation: Banawa, Cebu CityFloor Area: 90 square metersTotal Number of Bedrooms: 2Toilet and Bath: 2Car Parking: 1Turn-over condition: Clean and furnished excluding TV and internet connectionRental Rate: PhP35,000.00 per month 2-Bedroom furnished Apartment for rent with balcony located in Banawa, Cebu City. It is a quiet and gated compound and very accessible to Mc...',
  ['<i class="icon icon-bedroom"></i> : 2'],
  '₱ 35,000 / month\nNew listing'],
 ['Affordable Semi Furnished Apartment for rent in Banawa Cebu City, walking distance to main road.',
  'Cebu City\n                                    , Cebu',
  'Location: Banawa Cebu CityAffordable Semi Furnished Apartment for rent in Banawa Cebu City, near One Pavilion Mall. 2 minutes to One Pavilion Mall 5 minutes to Chong Hua Hospital15 minutes to Ayala mall Listing CB

In [3]:
# rows
df = pd.DataFrame(rows,
                  columns=['name','location','desc','accomodation','price'])

df

,name,location,desc,accomodation,price
0,2-Bedroom Furnished Apartment for Rent in Bana...,Cebu City\n ...,Property Type: Apartment for RentLocation: Ban...,"[<i class=""icon icon-bedroom""></i> : 2]","₱ 35,000 / month\nNew listing"
1,Affordable Semi Furnished Apartment for rent i...,Cebu City\n ...,Location: Banawa Cebu CityAffordable Semi Furn...,"[<i class=""icon icon-bedroom""></i> : 1]","₱ 12,000 / month\nNew listing"
2,Very Affordable Semi Furnished Apartment for r...,Cebu City\n ...,Location: Banawa Cebu CityVery Affordable Semi...,"[<i class=""icon icon-bedroom""></i> : 1]","₱ 12,000 / month\nNew listing"
3,2- Bedroom Apartment with Own Gate and Garage ...,"Angeles\n ,...","Rent/ Month: Php 35,000Details:90 sqm more or ...","[<i class=""icon icon-bedroom""></i> : 2]","₱ 35,000 / month"
4,Most Affordable 2 Bedroom Apartment for rent i...,Cebu City\n ...,Location: Banawa Cebu City2 Bedroom Apartment ...,"[<i class=""icon icon-bedroom""></i> : 2]","₱ 15,000 / month\nNew listing"
...,...,...,...,...,...
920,"LADY BEDSPACE FOR RENT IN AFPOVAI, FORT BONIFA...","Taguig\n , ...","LADY BEDSPACE FOR RENT IN AFPOVAI, FORT BONIFA...","[<i class=""icon icon-bedroom""></i> : 5]","₱ 2,500 / month"
921,Residential/Commercial Building for Lease at S...,Mandaluyong\n ...,Residential Building for Lease in Mandaluyong ...,"[<i class=""icon icon-bedroom""></i> : 43]","₱ 700,000 / month"
922,MACTAN LAPU-LAPU BOUGAINVILLEA Apartment 23 FO...,Lapu-Lapu\n ...,For Rent furnished 25 sqm studio type Bougainv...,"[<i class=""icon icon-bedroom""></i> : 1]","₱ 10,000 / month"
923,"1-Bedroom Semi-Furnished Apartment in Banawa, ...",Cebu City\n ...,Property Type: Apartment for RentLocation: nea...,"[<i class=""icon icon-bedroom""></i> : 1]","₱ 12,000 / month"


In [4]:
import datetime

today_d =  datetime.datetime.now().strftime('%m-%d-%y')
file_name = 'dotproperty-' + today_d + '.csv';
file_name


'dotproperty-03-05-23.csv'

In [5]:
from pathlib import Path  
filepath = Path(r'C:\Users\Jake\Desktop\YEJI\\3Y2S\Analytics Techniques and Tools\WEB_CRAWLING\\appartment_rental\\'+file_name)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)
